<a href="https://colab.research.google.com/github/jayant3297/Pre_Placement_Training/blob/main/15_July_DataWarehousing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TOPIC: Data Warehousing Fundamentals
   1. Design a data warehouse schema for a retail company that includes dimension tables for products, customers, and time. Implement the schema using a relational database management system (RDBMS) of your choice.
   2. Create a fact table that captures sales data, including product ID, customer ID, date, and sales amount. Populate the fact table with sample data.
   3. Write SQL queries to retrieve sales data from the data warehouse, including aggregations and filtering based on different dimensions.


Soltuion

1. Dimension tables:

Products: product_id (primary key), product_name, category, price
Customers: customer_id (primary key), customer_name, address, email
Time: date_id (primary key), date, day, month, year, quarter
Relational database management system (RDBMS): PostgreSQL

Creating a fact table and populating it with sample data:
Fact table:

Sales: sale_id (primary key), product_id (foreign key to Products), customer_id (foreign key to Customers), date_id (foreign key to Time), sales_amount, quantity_sold

Creating the fact table in PostgreSQL:

In [ ]:
CREATE TABLE sales (
    sale_id SERIAL PRIMARY KEY,
    product_id INTEGER,
    customer_id INTEGER,
    date_id INTEGER,
    sales_amount DECIMAL(10,2),
    quantity_sold INTEGER,

    CONSTRAINT fk_product FOREIGN KEY (product_id) REFERENCES products (product_id),
    CONSTRAINT fk_customer FOREIGN KEY (customer_id) REFERENCES customers (customer_id),
    CONSTRAINT fk_date FOREIGN KEY (date_id) REFERENCES time (date_id)
);


In [ ]:
INSERT INTO sales (product_id, customer_id, date_id, sales_amount, quantity_sold)
VALUES
    (1, 1, 1, 100.50, 2),
    (2, 1, 2, 50.75, 1),
    (3, 2, 1, 75.00, 3);

Writing SQL queries to retrieve sales data from the data warehouse:
Aggregation query to calculate total sales amount by product category:

In [ ]:
SELECT p.category, SUM(s.sales_amount) AS total_sales_amount
FROM sales s
JOIN products p ON s.product_id = p.product_id
GROUP BY p.category;


Filtering query to retrieve sales data for a specific customer:

In [ ]:
SELECT p.product_name, s.sales_amount, t.date
FROM sales s
JOIN products p ON s.product_id = p.product_id
JOIN time t ON s.date_id = t.date_id
WHERE s.customer_id = 1;


TOPIC: ETL and Data Integration
  1. Design an ETL process using a programming language (e.g., Python) to extract data from a source system (e.g., CSV files), transform it by applying certain business rules or calculations, and load it into a data warehouse.
  2. Implement the ETL process by writing code that performs the extraction, transformation, and loading steps.


Designing the ETL process:
Extraction: Extract data from CSV files.
Transformation: Apply business rules or calculations on the extracted data.
Loading: Load the transformed data into a data warehouse.

Implementing the ETL process in Python:

In [ ]:
import csv
import psycopg2

# Extract data from CSV files
def extract_data(file_path):
    data = []
    with open(file_path, 'r') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # Skip header row
        for row in csv_reader:
            data.append(row)
    return data

# Transform data by applying business rules or calculations
def transform_data(data):
    transformed_data = []
    for row in data:
        # Apply transformations
        transformed_row = [row[0], float(row[1]) * 1.1]  #increase value by 10%
        transformed_data.append(transformed_row)
    return transformed_data

# Load transformed data into a data warehouse (PostgreSQL)
def load_data(data):
    conn = psycopg2.connect(
        host='localhost',
        database='mydatabase',
        user='myuser',
        password='mypassword'
    )
    cur = conn.cursor()

# Create a table
    cur.execute('''
        CREATE TABLE IF NOT EXISTS transformed_data (
            id SERIAL PRIMARY KEY,
            field1 VARCHAR(255),
            field2 NUMERIC
        )
    ''')
    conn.commit()

# Insert data into the table
    for row in data:
        cur.execute('INSERT INTO transformed_data (field1, field2) VALUES (%s, %s)', row)
    conn.commit()

# Close the database connection
    cur.close()
    conn.close()



In [ ]:
# Main function
def etl_process():
    # Extract data from CSV files
    data = extract_data('data.csv')

    # Transform data
    transformed_data = transform_data(data)

    # Load transformed data into the data warehouse
    load_data(transformed_data)

    print("ETL completed.")

# Run the ETL process
etl_process()



TOPIC: Dimensional Modeling and Schemas
   1. Design a star schema for a university database, including a fact table for student enrollments and dimension tables for students, courses, and time. Implement the schema using a database of your choice.
   2. Write SQL queries to retrieve data from the star schema, including aggregations and joins between the fact table and dimension tables.


Designing a star schema for a university database:

Fact table:

Enrollments: enrollment_id (primary key), student_id (foreign key to Students), course_id (foreign key to Courses), date_id (foreign key to Time)

Dimension tables:

Students: student_id (primary key), student_name, student_major.

Courses: course_id (primary key), course_name, course_department.

Time: date_id (primary key), date, day, month, year.

Relational database management system (RDBMS): MySQL


In [ ]:
#creating tables

CREATE TABLE students (
    student_id INT PRIMARY KEY,
    student_name VARCHAR(255),
    student_major VARCHAR(255)
);

CREATE TABLE courses (
    course_id INT PRIMARY KEY,
    course_name VARCHAR(255),
    course_department VARCHAR(255)
);

CREATE TABLE time (
    date_id INT PRIMARY KEY,
    date DATE,
    day INT,
    month INT,
    year INT
);

CREATE TABLE enrollments (
    enrollment_id INT PRIMARY KEY,
    student_id INT,
    course_id INT,
    date_id INT,
    FOREIGN KEY (student_id) REFERENCES students (student_id),
    FOREIGN KEY (course_id) REFERENCES courses (course_id),
    FOREIGN KEY (date_id) REFERENCES time (date_id)
);


In [ ]:
#add data to the table

INSERT INTO students (student_id, student_name, student_major)
VALUES
    (1, 'John Doe', 'Computer Science'),
    (2, 'Jane Smith', 'Mathematics'),
    (3, 'Alice Johnson', 'Physics');


INSERT INTO courses (course_id, course_name, course_department)
VALUES
    (1, 'Introduction to Computer Science', 'Computer Science'),
    (2, 'Calculus I', 'Mathematics'),
    (3, 'Physics 101', 'Physics');


INSERT INTO time (date_id, date, day, month, year)
VALUES
    (1, '2022-01-01', 1, 1, 2022),
    (2, '2022-01-02', 2, 1, 2022),
    (3, '2022-01-03', 3, 1, 2022);



In [ ]:
#Writing SQL queries to retrieve data from the star schema:
#Aggregation query to calculate the total number of enrollments by course

SELECT c.course_name, COUNT(*) AS total_enrollments
FROM enrollments e
JOIN courses c ON e.course_id = c.course_id
GROUP BY c.course_name;


In [ ]:
#Join query to retrieve the student name, course name, and enrollment date:

SELECT s.student_name, c.course_name, t.date
FROM enrollments e
JOIN students s ON e.student_id = s.student_id
JOIN courses c ON e.course_id = c.course_id
JOIN time t ON e.date_id = t.date_id;


TOPIC: Performance Optimization and Querying

    1. Scenario: You need to improve the performance of your data loading process in the data warehouse. Write a Python script that implements the following optimizations:

      a)Utilize batch processing techniques to load data in bulk instead of individual row insertion.

      b)  Implement multi-threading or multiprocessing to parallelize the data loading process.
      c)  Measure the time taken to load a specific amount of data before and after implementing these optimizations.


In [ ]:
#Python script that implements the mentioned optimizations to improve the performance of the data loading process in a data warehouse

import time
import threading
from multiprocessing import Pool

# Function to simulate data loading process
def load_data(data)
    time.sleep(0.1)  # Simulating a delay of 0.1 seconds

# Function to load data in bulk using batch processing
def load_data_in_bulk(data, batch_size):
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        load_data(batch)

# Function to load data in parallel using multi-threading
def load_data_multi_threading(data, num_threads):
    threads = []
    for i in range(num_threads):
        thread = threading.Thread(target=load_data, args=(data,))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

# Function to load data in parallel using multiprocessing
def load_data_multiprocessing(data, num_processes):
    pool = Pool(processes=num_processes)
    pool.map(load_data, [data] * num_processes)
    pool.close()
    pool.join()

# Measure the time taken to load data
def measure_loading_time(data, num_iterations):
    start_time = time.time()

    for i in range(num_iterations):
        # Call the optimized data loading function here
        load_data_in_bulk(data, batch_size=1000)
        # load_data_multi_threading(data, num_threads=4)
        # load_data_multiprocessing(data, num_processes=4)

    end_time = time.time()
    loading_time = end_time - start_time

    return loading_time

# Generate sample data for loading
data = [i for i in range(10000)]  # Sample data to be loaded
num_iterations = 10  # Number of times to repeat the loading process

# Measure the loading time before implementing optimizations
loading_time_before = measure_loading_time(data, num_iterations)

# Measure the loading time after implementing optimizations
loading_time_after = measure_loading_time(data, num_iterations)

# Print the loading times
print("Loading time before optimization:", loading_time_before)
print("Loading time after optimization:", loading_time_after)
